## Day 23

https://adventofcode.com/2023/day/23

In [ ]:
def readInput23(infile):
    with open(infile) as f:
        return f.read().strip().splitlines()

### Part 1

BFS to map all paths and choose longest.

Also implemented modification to brute-force Part 2, but I have a better idea to simplfy the search...

In [3]:
from queue import Queue

def find_max_path(grid,S=(1,0),E=(len(grid[0])-2,len(grid)-1),part=1):
    q = Queue()
    q.put((S,set()))
    paths = []
    while not q.empty():
        p,path = q.get()
        if p==E:
            paths.append(len(path))
            continue
        x,y = p
        dirs = [(-1,0),(+1,0),(0,-1),(0,+1)]
        if part==1:
            if grid[y][x]=="v": dirs = [(0,+1)] 
            if grid[y][x]=="^": dirs = [(0,-1)] 
            if grid[y][x]==">": dirs = [(+1,0)] 
            if grid[y][x]=="<": dirs = [(-1,0)] 
        for dx,dy in dirs:
            xn,yn = (x+dx,y+dy)
            if xn<0 or xn>=len(grid[0]) or yn<0 or yn>=len(grid):
                continue
            if grid[yn][xn]=="#":
                continue
            pnew = (xn,yn)
            if pnew in path:
                continue
            else:
                pathnew = set(path)
                pathnew.add(pnew)
                q.put((pnew,pathnew))
    return max(paths)

In [4]:
def part1(infile):
    grid = readInput23(infile)
    S = (1,0)
    E = (len(grid[0])-2,len(grid)-1)
    return find_max_path(grid,S,E)

In [5]:
print("Test 1:",part1("examples/example23.txt"))

Test 1: 94


In [6]:
part1("examples/input23.txt")

2074

### Part 2

Mapping all the junction nodes, their connection and their distances to simplify the problem, then using `NetworkX` to find all path in the reduced graph:

In [49]:
from queue import Queue

def map_nodes(grid):
    # find nodes in grid
    nodes = [(1,0),(len(grid[0])-2,len(grid)-1)]
    for x in range(1,len(grid[0])-1):
        for y in range(1,len(grid)-1):
            if grid[y][x]!="#":
                n = [(x+dx,y+dy) for dx,dy in [(-1,0),(+1,0),(0,-1),(0,+1)] if grid[y+dy][x+dx]!="#"]
                if len(n)>2:
                    nodes.append((x,y))
    # map connections and distances between nodes
    nodemap = {}
    for n in nodes:
        paths = {}
        q = Queue()
        q.put((n,{n}))
        while not q.empty():
            p,path = q.get()
            if p!=n and p in nodes and len(path):
                paths[p] = len(path)-1
                continue
            x,y = p
            for dx,dy in [(-1,0),(+1,0),(0,-1),(0,+1)]:
                xn,yn = (x+dx,y+dy)
                if xn<0 or xn>=len(grid[0]) or yn<0 or yn>=len(grid):
                    continue
                if grid[yn][xn]=="#":
                    continue
                pnew = (xn,yn)
                if pnew in path:
                    continue
                else:
                    pathnew = set(path)
                    pathnew.add(pnew)
                    q.put((pnew,pathnew))
        nodemap[n] = paths
    return nodemap

In [79]:
# Using NetworkX to solve the graph, looking for all simple paths from S to E and compting their lenghts

In [75]:
import networkx as nx

def part2(infile):
    grid = readInput23(infile)
    S = (1,0)
    E = (len(grid[0])-2,len(grid)-1)
    nodemap = map_nodes(grid)
    G = nx.Graph()
    for n,c in nodemap.items():
        for m,d in c.items():
            G.add_edge(n,m,weight=d)
    paths = []
    for p in nx.all_simple_paths(G,S,E):
        l = sum([ nodemap[p[i]][p[i-1]] for i in range (1,len(p)) ])
        paths.append(l)
    return max(paths)

In [76]:
part2("examples/example23.txt")

154

In [77]:
part2("examples/input23.txt")

6494